## Training ML models


Once you have explored the repository and selected an appropriate training dataset, you can download it and train a model. In this notebook we will show an example of how to do so using the [EuroSAT](https://www.eotdl.com/datasets/EuroSAT-RGB) dataset, in its RGB version.


In [ ]:
from eotdl.datasets import stage_dataset

path = stage_dataset("EuroSAT-RGB", version=1, path="data", force=True, assets=True)
path

In [ ]:
!unzip {path}/EuroSAT-RGB.zip -d data/train

> This might take a few minutes!


In order to streamline the training process, we will use the [PytorchEO](https://github.com/earthpulse/pytorchEO) library. This open source library is built on top of [Pytorch](https://pytorch.org/) and [Pytorch Lightning](https://lightning.ai/) to facilitate the design, implementation, training and deployment of deep learning models for Earth Observation. It offers AI-Ready EO datasets as well as ready-to-use tasks and models.

It is not our goal to provide a complete tutorial on how to train a model, but rather to show how to use the EOTDL. If you want to learn more about AI and training deep neural networks, we encourage you to explore the [PytorchEO](https://github.com/earthpulse/pytorchEO) library, and even contribute with more datasets, tasks, models and wrappers. We challenge you to train a better model!


In [ ]:
from pytorch_eo.datasets import EuroSATRGB

# do not set download to True since it will download the dataset from the original source and not EOTDL !

ds = EuroSATRGB(
    batch_size=25,
    verbose=True,
    path="data",
    download=False,
    data_folder="train/EuroSAT-RGB",
)

ds.setup()

In [ ]:
ds.df

This dataset contains 27000 Sentinel 2 images classified in 10 categories.


ds.num_classes, ds.classes


import matplotlib.pyplot as plt

batch = next(iter(ds.train_dataloader()))
imgs, labels = batch['image'], batch['label']

fig = plt.figure(figsize=(10, 10))
for i, (img, label) in enumerate(zip(imgs, labels)):
ax = plt.subplot(5, 5, i + 1)
ax.imshow(img.permute(1, 2, 0))
ax.set_title(ds.classes[label.item()])
ax.axis('off')
plt.tight_layout()
plt.show()


We will train a neural network for the task of Image classification.


In [ ]:
import torch
from pytorch_eo.tasks.classification import ImageClassification

task = ImageClassification(num_classes=ds.num_classes)

sample_inputs = torch.randn(8, 3, 224, 224)
output = task(sample_inputs)
output.shape

In [ ]:
import lightning as L

trainer = L.Trainer(
    accelerator="cuda",
    devices=1,
    max_epochs=10,
    enable_checkpointing=False,
    # limit_train_batches=50,
    # limit_val_batches=50,
)

trainer.fit(task, ds)

Once the model is trained you can evaluate it using some test data.


In [ ]:
from tqdm import tqdm

task.eval()
acc = 0
with torch.no_grad():
    for batch in tqdm(ds.test_dataloader()):
        output = task(batch["image"])
        acc += (output.argmax(1) == batch["label"]).sum().item()

print("test accuracy: ", f"{acc}/{len(ds.test_dataloader().dataset)}")

In [ ]:
# extract batch from test dataloader

batch = next(iter(ds.test_dataloader(shuffle=True, batch_size=25)))
imgs, labels = batch["image"], batch["label"]

# compute predictions

preds = task.predict(batch)
preds = torch.argmax(preds, axis=1)

# visualize predictions

fig = plt.figure(figsize=(10, 10))
for i, (img, label, pred) in enumerate(zip(imgs, labels, preds)):
    ax = plt.subplot(5, 5, i + 1)
    ax.imshow(img.permute(1, 2, 0))
    gt = ds.classes[label.item()]
    pred = ds.classes[pred.item()]
    ax.set_title(gt, color="green" if gt == pred else "red")
    ax.axis("off")
plt.tight_layout()
plt.show()

And export it to later ingestion to the EOTDL. You can choose your preferred export method, here we use ONNX.


In [ ]:
# !pip install onnx onnxruntime

In [ ]:
import os

filepath = "data/EuroSAT-RGB-model/model.onnx"
os.makedirs(os.path.dirname(filepath), exist_ok=True)

task.to_onnx(
    filepath,
    imgs,
    export_params=True,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch_size", 2: "height", 3: "width"},
        "output": {0: "batch_size"},
    },
)

In [ ]:
import onnxruntime
import numpy as np

x = np.random.randn(3, 3, 132, 412).astype(np.float32)

ort_session = onnxruntime.InferenceSession(filepath)
input_name = ort_session.get_inputs()[0].name
ort_inputs = {input_name: x}
ort_outs = ort_session.run(None, ort_inputs)
ort_outs[0].shape

In [ ]:
acc = 0
for batch in tqdm(ds.test_dataloader()):
    ort_inputs = {input_name: batch["image"].numpy()}
    ort_outs = ort_session.run(None, ort_inputs)
    acc += np.sum((ort_outs[0].argmax(1) == batch["label"].numpy()))

print("test accuracy: ", f"{acc}/{len(ds.test_dataloader().dataset)}")

## Discussion and Contribution opportunities


Feel free to ask questions now (live or through Discord) and make suggestions for future improvements.

- What would you like to see in the EOTDL concerning training?
- What are the main challenges you face when training ML models with EO data?
- What are the main datasets you would like to see in the EOTDL?
- What are the main tasks you would like to see implemented?
- What are the main models you would like to see implemented?
- What are the frameworks you would like to have wrappers for?
